# Energy Consumption Forecasting with Darts

This notebook implements a comprehensive time series forecasting project for predicting energy consumption in the PJM East region using multiple models.

**Reference Documentation:** [darts.example.md](darts.example.md)

**Citations:**
- Darts Library: https://unit8co.github.io/darts/
- Herzen et al. (2022). "Darts: User-Friendly Modern Machine Learning for Time Series" JMLR 23(124):1−6
- Dataset: https://www.kaggle.com/datasets/robikscube/hourly-energy-consumption

**Notebook Flow:**
1. Setup and Imports
2. Data Ingestion
3. Exploratory Data Analysis
4. Feature Engineering
5. Model Comparison (Prophet, N-BEATS, LSTM)
6. Hyperparameter Tuning
7. Final Evaluation and Visualization


## 1. Setup and Imports


In [ ]:
import logging
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import ParameterGrid
from tqdm import tqdm

# Darts core imports.
from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler
from darts.utils.statistics import check_seasonality
from darts.metrics import mape, rmse, mae, smape
from darts.utils.utils import ModelMode

# Darts models.
from darts.models import (
    Prophet,
    NBEATSModel,
    RNNModel,
    ExponentialSmoothing,
    NaiveSeasonal
)

# Local utility functions.
import darts_utils as utils

# Configure logging.
logging.basicConfig(level=logging.INFO)
_LOG = logging.getLogger(__name__)

# Set plotting style.
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 12

# Set random seed for reproducibility.
np.random.seed(42)

print("✅ All imports successful!")


## 2. Data Ingestion

Load the PJME Hourly Energy Consumption dataset and prepare it for time series analysis.


In [ ]:
# Load the PJME hourly energy consumption data using utility function.
data_path = 'PJME_hourly.csv'
df = utils.load_energy_data(data_path)

# Display basic info.
print(f"📊 Dataset Shape: {df.shape}")
print(f"📅 Date Range: {df.index.min()} to {df.index.max()}")
print(f"📈 Total Hours: {len(df):,}")
df.head(10)


In [ ]:
# Handle missing timestamps with interpolation.
df = utils.handle_missing_timestamps(df)

# Display statistics.
df.describe()


In [ ]:
# Create Darts TimeSeries object.
series = utils.create_darts_series(df)

# Use last 3 years of data for faster training.
YEARS_OF_DATA = 3
series_subset = series[-24*365*YEARS_OF_DATA:]

print(f"✅ TimeSeries created!")
print(f"📊 Using last {YEARS_OF_DATA} years: {len(series_subset)} hours")
print(f"📅 Start: {series_subset.start_time()}")
print(f"📅 End: {series_subset.end_time()}")


## 3. Exploratory Data Analysis

Analyze the time series for patterns, seasonality, and distribution.


In [ ]:
# Plot the time series.
utils.plot_time_series(
    series_subset,
    title='PJME Energy Consumption (Last 3 Years)'
)


In [ ]:
# Analyze seasonality patterns using utility function.
utils.plot_seasonality_analysis(df)


In [ ]:
# Check for seasonality using statistical tests.
print("🔍 Seasonality Analysis:")
print("=" * 50)

is_daily, _ = check_seasonality(series_subset, m=24, max_lag=48)
print(f"Daily (24h) seasonality detected: {is_daily}")

is_weekly, _ = check_seasonality(series_subset, m=168, max_lag=336)
print(f"Weekly (168h) seasonality detected: {is_weekly}")


## 4. Feature Engineering

Create temporal features, lag values, and rolling statistics to improve model performance.


In [ ]:
# Create temporal features.
df_features = utils.create_temporal_features(df)
print("✅ Temporal features created!")
df_features.head()


In [ ]:
# Add lag features.
df_features = utils.add_lag_features(df_features)
print("✅ Lag features created!")

# Add rolling features.
df_features = utils.add_rolling_features(df_features)
print("✅ Rolling features created!")

# Display all features.
print(f"\n📊 Total features: {len(df_features.columns)}")
print(f"Columns: {df_features.columns.tolist()}")


In [ ]:
# Feature correlation analysis.
fig, ax = plt.subplots(figsize=(14, 10))

numeric_cols = df_features.select_dtypes(include=[np.number]).columns
corr_matrix = df_features[numeric_cols].dropna().corr()

# Plot correlation with target.
target_corr = corr_matrix['energy_consumption'].drop('energy_consumption').sort_values()
colors = ['red' if x < 0 else 'green' for x in target_corr.values]
target_corr.plot(kind='barh', color=colors, ax=ax)
ax.set_title('Feature Correlation with Energy Consumption', fontsize=16, fontweight='bold')
ax.set_xlabel('Correlation Coefficient')
ax.axvline(x=0, color='black', linestyle='-', linewidth=0.5)
plt.tight_layout()
plt.show()


## 5. Model Comparison

Compare multiple forecasting models: Naive Seasonal, Exponential Smoothing, Prophet, N-BEATS, and LSTM.


In [ ]:
# Configure train/test split.
TEST_SIZE = 24 * 30  # 30 days for testing.
FORECAST_HORIZON = 24 * 7  # 7 days forecast.

# Split data using utility function.
train, test = utils.train_test_split_series(series_subset, test_size=TEST_SIZE)

# Scale data for neural network models.
train_scaled, test_scaled, scaler = utils.scale_series(train, test)

print(f"📊 Training set: {len(train)} hours ({len(train)/24:.0f} days)")
print(f"📊 Test set: {len(test)} hours ({len(test)/24:.0f} days)")
print(f"🎯 Forecast horizon: {FORECAST_HORIZON} hours ({FORECAST_HORIZON/24:.0f} days)")


In [ ]:
# Visualize train/test split.
fig, ax = plt.subplots(figsize=(16, 6))
train.plot(ax=ax, label='Training Data', color='blue')
test.plot(ax=ax, label='Test Data', color='orange')
ax.set_title('Train/Test Split', fontsize=16, fontweight='bold')
ax.set_xlabel('Date')
ax.set_ylabel('Energy (MW)')
ax.legend(loc='upper right')
plt.tight_layout()
plt.show()


In [ ]:
# Dictionary to store model results.
model_results = {}


### 5.1 Naive Seasonal Model (Baseline)


In [ ]:
# Train Naive Seasonal model with weekly seasonality.
print("🔄 Training Naive Seasonal Model...")
naive_model = NaiveSeasonal(K=168)
naive_model.fit(train)
naive_pred = naive_model.predict(len(test))

# Evaluate using utility function.
metrics = utils.evaluate_forecast(test, naive_pred, "Naive Seasonal")
model_results['Naive Seasonal'] = {'predictions': naive_pred, **metrics}
print("✅ Naive Seasonal Model trained!")


### 5.2 Exponential Smoothing


In [ ]:
# Train Exponential Smoothing with daily seasonality.
print("🔄 Training Exponential Smoothing Model...")
exp_model = ExponentialSmoothing(seasonal_periods=24, trend=None, seasonal=ModelMode.ADDITIVE)
exp_model.fit(train)
exp_pred = exp_model.predict(len(test))

# Evaluate.
metrics = utils.evaluate_forecast(test, exp_pred, "Exponential Smoothing")
model_results['Exponential Smoothing'] = {'predictions': exp_pred, **metrics}
print("✅ Exponential Smoothing Model trained!")


### 5.3 Prophet Model


In [ ]:
# Train Prophet with multiple seasonalities.
print("🔄 Training Prophet Model...")
print("   (This may take a few minutes...)")
prophet_model = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=True,
    daily_seasonality=True,
    seasonality_mode='multiplicative'
)
prophet_model.fit(train)
prophet_pred = prophet_model.predict(len(test))

# Evaluate.
metrics = utils.evaluate_forecast(test, prophet_pred, "Prophet")
model_results['Prophet'] = {'predictions': prophet_pred, **metrics}
print("✅ Prophet Model trained!")


In [ ]:
# Train N-BEATS model.
print("🔄 Training N-BEATS Model...")
print("   (This may take several minutes...)")
nbeats_model = NBEATSModel(
    input_chunk_length=168,
    output_chunk_length=24,
    generic_architecture=True,
    num_stacks=10,
    num_blocks=1,
    num_layers=4,
    batch_size=1024,
    layer_widths=256,
    n_epochs=50,
    random_state=42,
    pl_trainer_kwargs={"enable_progress_bar": True, "accelerator": "auto"}
)
nbeats_model.fit(train_scaled, verbose=True)

# Predict and inverse transform.
nbeats_pred_scaled = nbeats_model.predict(len(test))
nbeats_pred = scaler.inverse_transform(nbeats_pred_scaled)

# Evaluate.
metrics = utils.evaluate_forecast(test, nbeats_pred, "N-BEATS")
model_results['N-BEATS'] = {'predictions': nbeats_pred, **metrics}
print("✅ N-BEATS Model trained!")


### 5.5 LSTM Model (Deep Learning)


In [ ]:
# Train LSTM model.
print("🔄 Training LSTM Model...")
print("   (This may take several minutes...)")
lstm_model = RNNModel(
    model='LSTM',
    input_chunk_length=168,
    output_chunk_length=24,
    training_length=192,
    hidden_dim=64,
    batch_size=1024,
    n_rnn_layers=2,
    dropout=0.1,
    n_epochs=50,
    random_state=42,
    pl_trainer_kwargs={"enable_progress_bar": True, "accelerator": "auto"}
)
lstm_model.fit(train_scaled, verbose=True)

# Predict and inverse transform.
lstm_pred_scaled = lstm_model.predict(len(test))
lstm_pred = scaler.inverse_transform(lstm_pred_scaled)

# Evaluate.
metrics = utils.evaluate_forecast(test, lstm_pred, "LSTM")
model_results['LSTM'] = {'predictions': lstm_pred, **metrics}
print("✅ LSTM Model trained!")


In [ ]:
# Create model comparison summary.
summary_df = utils.compare_models(model_results)

print("\n" + "=" * 60)
print("📊 MODEL COMPARISON SUMMARY")
print("=" * 60)
print(summary_df.to_string(index=False))

best_model_name = summary_df.iloc[0]['Model']
print(f"\n🏆 Best Model: {best_model_name}")


## 6. Hyperparameter Tuning

Optimize N-BEATS model using grid search and cross-validation.


In [ ]:
# Define parameter grid for N-BEATS.
param_grid = {
    'input_chunk_length': [168],
    'output_chunk_length': [24],
    'num_stacks': [5, 10],
    'num_layers': [2, 4],
    'layer_widths': [128, 256]
}

# Create validation set from training data.
VAL_SIZE = 24 * 7  # 1 week.
train_tune = train_scaled[:-VAL_SIZE]
val_tune = train_scaled[-VAL_SIZE:]

print(f"📊 Tuning train size: {len(train_tune)} hours")
print(f"📊 Validation size: {len(val_tune)} hours")
print(f"📊 Total parameter combinations: {len(list(ParameterGrid(param_grid)))}")


In [ ]:
# Perform grid search.
print("🔄 Running Grid Search...")
print("   (This may take a while...)\n")

tuning_results = []
for params in tqdm(list(ParameterGrid(param_grid))):
    try:
        model = NBEATSModel(
            input_chunk_length=params['input_chunk_length'],
            output_chunk_length=params['output_chunk_length'],
            generic_architecture=True,
            num_stacks=params['num_stacks'],
            num_blocks=1,
            num_layers=params['num_layers'],
            layer_widths=params['layer_widths'],
            n_epochs=20,
            batch_size=1024,
            random_state=42,
            pl_trainer_kwargs={"enable_progress_bar": False, "accelerator": "auto"}
        )
        model.fit(train_tune, verbose=False)
        pred = model.predict(len(val_tune))
        score = mape(val_tune, pred)
        params['mape'] = score
        tuning_results.append(params)
        print(f"   Params: {params} -> MAPE: {score:.2f}%")
    except Exception as e:
        print(f"   Error with params {params}: {e}")
        continue

print("\n✅ Grid Search Complete!")


In [ ]:
# Find best parameters.
tuning_df = pd.DataFrame(tuning_results).sort_values('mape')
print("📊 Hyperparameter Tuning Results:")
print("=" * 60)
print(tuning_df.to_string(index=False))

best_params = tuning_df.iloc[0].to_dict()
del best_params['mape']
print(f"\n🏆 Best Parameters:")
for key, value in best_params.items():
    print(f"   {key}: {value}")


In [ ]:
# Train optimized N-BEATS model with best parameters.
print("\n🔄 Training Optimized N-BEATS Model...")
optimized_nbeats = NBEATSModel(
    input_chunk_length=int(best_params['input_chunk_length']),
    output_chunk_length=int(best_params['output_chunk_length']),
    generic_architecture=True,
    num_stacks=int(best_params['num_stacks']),
    num_blocks=1,
    num_layers=int(best_params['num_layers']),
    layer_widths=int(best_params['layer_widths']),
    n_epochs=200,
    batch_size=1024,
    random_state=42,
    pl_trainer_kwargs={"enable_progress_bar": True, "accelerator": "auto"}
)
optimized_nbeats.fit(train_scaled, verbose=True)

# Predict and evaluate.
optimized_pred_scaled = optimized_nbeats.predict(len(test))
optimized_pred = scaler.inverse_transform(optimized_pred_scaled)
metrics = utils.evaluate_forecast(test, optimized_pred, "Optimized N-BEATS")
model_results['Optimized N-BEATS'] = {'predictions': optimized_pred, **metrics}
print("\n✅ Optimized N-BEATS Model trained!")


## 7. Final Evaluation and Visualization

Analyze model performance and visualize predictions across different time windows.


In [ ]:
# Final model comparison summary.
final_df = utils.compare_models(model_results)

print("\n" + "=" * 70)
print("📊 FINAL MODEL COMPARISON")
print("=" * 70)
print(final_df.to_string(index=False))

best_model_name = final_df.iloc[0]['Model']
print(f"\n🏆 BEST MODEL: {best_model_name}")
print(f"   MAPE: {final_df.iloc[0]['MAPE (%)']}%")


In [ ]:
predictions_dict = {name: results['predictions'] for name, results in model_results.items()}

# Corrected plotting logic, inlining the functionality of plot_predictions_vs_actual
fig, ax = plt.subplots(figsize=(16, 6))

# Plot actual data
test.plot(ax=ax, label='Actual', color='black', linewidth=2)

# Plot predictions for each model
colors = plt.cm.tab10(np.linspace(0, 1, len(predictions_dict)))
for (model_name, pred), color in zip(predictions_dict.items(), colors):
    # Convert numpy array color to a tuple for darts plot function
    pred.plot(ax=ax, label=model_name, color=tuple(color), linewidth=1.5, alpha=0.8)

ax.set_title('All Model Predictions vs Actual', fontsize=16, fontweight='bold')
ax.set_xlabel('Date')
ax.set_ylabel('Energy (MW)')
ax.legend(loc='upper right')
plt.tight_layout()
plt.show()

In [ ]:
# Detailed view: Best model prediction vs actual by week.
best_predictions = model_results[best_model_name]['predictions']

fig, axes = plt.subplots(4, 1, figsize=(16, 16))
weeks = [(0, 168, 'Week 1'), (168, 336, 'Week 2'), (336, 504, 'Week 3'), (504, 672, 'Week 4')]

for idx, (start, end, week_name) in enumerate(weeks):
    ax = axes[idx]
    actual_week = test[start:end]
    pred_week = best_predictions[start:end]
    actual_week.plot(ax=ax, label='Actual', color='black', linewidth=2)
    pred_week.plot(ax=ax, label=f'{best_model_name}', color='green', linewidth=2, alpha=0.8)
    weekly_mape = mape(actual_week, pred_week)
    ax.set_title(f'{week_name} - MAPE: {weekly_mape:.2f}%', fontsize=14, fontweight='bold')
    ax.set_xlabel('Date')
    ax.set_ylabel('Energy (MW)')
    ax.legend(loc='upper right')
    ax.fill_between(
        actual_week.time_index,
        actual_week.univariate_values().flatten(),
        pred_week.univariate_values().flatten(),
        alpha=0.3, color='red'
    )

plt.tight_layout()
plt.savefig('best_model_weekly.png', dpi=150, bbox_inches='tight')
plt.show()
print("📁 Saved: best_model_weekly.png")


In [ ]:
# Error analysis using utility function.
utils.plot_error_analysis(test, best_predictions)


In [ ]:
# Generate future forecast.
print("🔮 Generating Future Forecast...")

# Train on all available data.
full_train_scaled = scaler.fit_transform(series_subset)
future_model = NBEATSModel(
    input_chunk_length=int(best_params.get('input_chunk_length', 168)),
    output_chunk_length=int(best_params.get('output_chunk_length', 24)),
    generic_architecture=True,
    num_stacks=int(best_params.get('num_stacks', 10)),
    num_blocks=1,
    num_layers=int(best_params.get('num_layers', 4)),
    layer_widths=int(best_params.get('layer_widths', 256)),
    n_epochs=50,
    batch_size=1024,
    random_state=42,
    pl_trainer_kwargs={"enable_progress_bar": True, "accelerator": "auto"}
)
future_model.fit(full_train_scaled, verbose=True)

# Predict next 7 days.
FUTURE_HORIZON = 24 * 7
future_pred_scaled = future_model.predict(FUTURE_HORIZON)
future_pred = scaler.inverse_transform(future_pred_scaled)
print(f"\n✅ Generated {FUTURE_HORIZON} hour forecast!")


In [ ]:
# Plot future forecast.
fig, ax = plt.subplots(figsize=(16, 6))
historical = series_subset[-24*14:]
historical.plot(ax=ax, label='Historical Data', color='blue', linewidth=2)
future_pred.plot(ax=ax, label='7-Day Forecast', color='red', linewidth=2)
ax.set_title('7-Day Energy Consumption Forecast', fontsize=16, fontweight='bold')
ax.set_xlabel('Date')
ax.set_ylabel('Energy (MW)')
ax.legend(loc='upper right')
ax.axvline(x=series_subset.end_time(), color='green', linestyle='--', linewidth=2)
plt.tight_layout()
plt.savefig('future_forecast.png', dpi=150, bbox_inches='tight')
plt.show()
print("📁 Saved: future_forecast.png")


## Summary

### Key Findings

1. **Seasonality**: The PJME energy data shows strong daily, weekly, and yearly seasonal patterns.
2. **Model Performance**: Deep learning models (N-BEATS, LSTM) generally outperform traditional statistical methods.
3. **Best Model**: The optimized N-BEATS model achieved the best performance after hyperparameter tuning.
4. **Feature Importance**: Temporal features (hour of day, day of week) are highly correlated with energy consumption.

### Recommendations

1. For production deployment, consider ensemble methods combining multiple models.
2. Incorporate external features like weather data for improved accuracy.
3. Regularly retrain models as consumption patterns may shift over time.
4. Monitor model performance and implement automated retraining pipelines.


In [ ]:
# Save final results.
import json

final_df.to_csv('model_results.csv', index=False)
print("📁 Saved: model_results.csv")

with open('best_params.json', 'w') as f:
    json.dump({k: int(v) if isinstance(v, (int, np.integer)) else v
               for k, v in best_params.items()}, f, indent=2)
print("📁 Saved: best_params.json")

print("\n" + "=" * 60)
print("✅ ENERGY FORECASTING PROJECT COMPLETE!")
print("=" * 60)
